# Test Clustertools with AMUSE data

First, make sure the packages are installed / up-to-date

In [ ]:
! pip install --upgrade pip
! pip install --upgrade amuse-framework
! pip install --upgrade clustertools

Import packages

In [ ]:
import numpy as np
import clustertools as ct
from amuse.units import units, nbody_system, constants
from amuse.io import read_set_from_file
from amuse.support.console import set_printing_strategy
from amuse.ic.plummer import new_plummer_model
from amuse.ic.kroupa import new_kroupa_mass_distribution
set_printing_strategy('custom', preferred_units=[units.pc, units.Myr, units.kms])

Create a star cluster model

In [ ]:
number_of_stars = 3000
masses = new_kroupa_mass_distribution(number_of_stars)
cluster_radius = 2 | units.pc
converter = nbody_system.nbody_to_si(cluster_radius, masses.sum())
stars_initial = new_plummer_model(number_of_stars, converter)
stars_initial.mass = masses

Create a galactic potential model to put the cluster in orbit of

In [ ]:
class SimpleGalacticModel:
    def __init__(self, radius=1 | units.kpc, mass=1.5e10 | units.MSun, alpha=1.2):
        self.radius = radius
        self.mass = mass
        self.alpha = alpha

    def get_gravity_at_point(self, epsilon, x, y, z):
        r2 = x**2 + y**2 + z**2 + epsilon**2
        r = r2**0.5
        m = self.mass * (r/self.radius)**self.alpha
        fr = constants.G * m / r2
        ax = -fr * x / r
        ay = -fr * y / r
        az = -fr * z / r
        return ax, ay, az

    def circular_velocity(self, distance):
        m = self.mass * (distance / self.radius)**self.alpha
        vc = (constants.G * m/distance)**0.5
        return vc

    def get_potential_at_point(self, epsilon, x, y, z):
        r2 = x**2 + y**2 + z**2 + epsilon**2
        r = r2**0.5
        c = constants.G * self.mass / self.radius**self.alpha
        phi = c / (self.alpha-1) * (r**(self.alpha-1) - self.radius**(self.alpha-1))
        return phi

Put the cluster in orbit

In [ ]:
galaxy = SimpleGalacticModel()
cluster_distance = 100 | units.pc
stars_initial.x += cluster_distance
stars_initial.vy += 0.8 * galaxy.circular_velocity(cluster_distance)

Manually add stars to ClusterTools (keeping units)

In [ ]:
cluster = ct.StarCluster(ctype='amuse', origin='galaxy')
s = stars_initial
cluster.add_stars(s.x, s.y, s.z, s.vx, s.vy, s.vz, s.mass, s.key)

cluster.analyze(sortstars=True)

**starplot cannot handle AMUSE units, so the cell below fails**

In [ ]:
ct.starplot(cluster)

Alternative: directly load AMUSE cluster (this seems to get rid of the AMUSE units)

In [ ]:
cluster_direct = ct.load_cluster('amuse', particles=stars_initial, units='pckms', origin='galaxy')
cluster_direct.analyze(sortstars=True)
ct.starplot(cluster_direct)

In [ ]:
# cluster.add_orbit(xgc,ygc,zgc,vxgc,vygc,vzgc)

In [ ]:
def ct_vs_amuse(
    ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz,
    am_x, am_y, am_z, am_vx, am_vy, am_vz,
):
    print(
        f"Clustertools finds position/velocity: {ct_x}, {ct_y}, {ct_z}, {ct_vx}, {ct_vy}, {ct_vz}"
    )
    print(
        f"Difference in position (vs center-of-mass): {ct_x-am_x}, {ct_y-am_y}, {ct_z-am_z}\n"
        f"Difference in velocity (vs center-of-mass): {ct_vx-am_vx}, {ct_vy-am_vy}, {ct_vz-am_vz}\n"
    )

In [ ]:
ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz = cluster.find_centre()
am_x, am_y, am_z = stars_initial.center_of_mass()
am_vx, am_vy, am_vz = stars_initial.center_of_mass_velocity()
dc, cr, crho = stars_initial.densitycentre_coreradius_coredens(unit_converter=converter)
am_dx, am_dy, am_dz = dc
print("CT vs AMUSE center-of-mass:")
ct_vs_amuse(
    ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz,
    am_x, am_y, am_z, am_vx, am_vy, am_vz,
)
print("CT vs AMUSE density center (with Hop, ignore the velocities):")
ct_vs_amuse(
    ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz,
    am_dx, am_dy, am_dz, am_vx, am_vy, am_vz,
)

**Weird / bug: running the cell above a second time finds the cluster at the origin?**

(this problem does not occur when using the code in the cell below)

In [ ]:
ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz = ct.find_centre(cluster)
try:
    print(f"Positions are in unit {ct_x.unit}")
except:
    print(f"Not using units - assuming pc/kms")
    ct_x , ct_y, ct_z, ct_vx, ct_vy, ct_vz = ct_x | units.pc, ct_y | units.pc, ct_z | units.pc, ct_vx | units.kms, ct_vy | units.kms, ct_vz | units.kms,

print("CT vs AMUSE center-of-mass:")
ct_vs_amuse(
    ct_x , ct_y, ct_z, ct_vx, ct_vy, ct_vz,
    am_x, am_y, am_z, am_vx, am_vy, am_vz,
)
print("CT vs AMUSE density center (with Hop, ignore the velocities):")
ct_vs_amuse(
    ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz,
    am_dx, am_dy, am_dz, am_vx, am_vy, am_vz,
)

**Using the center-of-mass in Clustertools seems to ignore the galactic coordinates?**

In [ ]:
# alternatively, find the center-of-mass rather than density center:
ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz = cluster.find_centre(density=False)
print("CT center-of-mass vs AMUSE center-of-mass:")
ct_vs_amuse(
    ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz,
    am_x, am_y, am_z, am_vx, am_vy, am_vz,
)
print("CT center-of-mass vs AMUSE density center (with Hop, ignore the velocities):")
ct_vs_amuse(
    ct_x, ct_y, ct_z, ct_vx, ct_vy, ct_vz,
    am_dx, am_dy, am_dz, am_vx, am_vy, am_vz,
)

In [ ]:
print('Half-Mass Relaxation Time: ',cluster.half_mass_relaxation_time())
print('Core Relaxation Time: ',cluster.core_relaxation_time())
print('Lagrange Radii: ',cluster.rlagrange())
print('Virial Radius: ',cluster.virial_radius())

- Why is this result slightly different than that of cluster? It's the same data...
- also, the AMUSE-fed data has units?

In [ ]:
ct.skyplot(cluster)

In [ ]:
ct.skyplot(cluster2)

- Clustertools seems to have some trouble with AMUSE units still. I wonder how it deals with Astropy units?

In [ ]:
print('Mean radius with numpy =', np.mean(cluster.r))
print('Mean radius =', cluster.rmean)

print('Half mass radius =', cluster.rm)

cluster.rlagrange()
print('50% Lagrange Radius = ',cluster.rn[4])

In [ ]:
print('Mean radius with numpy =', np.mean(cluster2.r))
print('Mean radius =', cluster2.rmean)

print('Half mass radius =', cluster2.rm)

cluster2.rlagrange()
print('50% Lagrange Radius = ',cluster2.rn[4])


- this doesn't seem to take the cluster's origin into account

In [ ]:
cluster.to_kpckms()
print('Mean radius with numpy = ',np.mean(cluster.r))
print('Mean radius with clustertools = ',cluster.rmean)

print('Half mass radius =', cluster.rm)
cluster.rlagrange()
print('50% Lagrange Radius = ',cluster.rn[4])

In [ ]:
cluster2.to_kpckms()
print('Mean radius with numpy = ',np.mean(cluster2.r))
print('Mean radius with clustertools = ',cluster2.rmean)

print('Half mass radius =', cluster2.rm)
cluster2.rlagrange()
print('50% Lagrange Radius = ',cluster2.rn[4])

- to_kpckms() erases units and moves cluster to origin?
- still a difference between the two clusters?

In [ ]:
cluster.reset_nbody_scale()
print('MASS SCALING: ',cluster.zmbar)
print('POSITION SCALING: ',cluster.rbar)
print('VELOCITY SCALING: ',cluster.vbar)
print('TIME SCALING: ',cluster.tbar)

- cluster has 2000 stars so this seems logical

In [ ]:
cluster.to_nbody()
ct.starplot(cluster)

In [ ]:
cluster.to_radec()
ct.starplot(cluster)

In [ ]:
#At the moment, our cluster is not perfectly in virial equilibrium:
cluster.energies()
print(cluster.qvir)
#However it can be scaled such that ``cluster.qvir=-0.5``
cluster.virialize()
cluster.energies()

print('New Qv: ',cluster.qvir)


In [ ]:
#At the moment, our cluster is not perfectly in virial equilibrium:
cluster2.energies()
print(cluster2.qvir)
#However it can be scaled such that ``cluster.qvir=-0.5``
cluster2.virialize()
cluster2.energies()

print('New Qv: ',cluster2.qvir)


- huh? this doesn't look correct...

In [ ]:
#How many stars have vtheta < 0 in the clustercentric coordinate system
cluster.to_cluster()
r, theta, z, vr, vtheta, vz = ct.cart_to_cyl(cluster.x,cluster.y,cluster.z,cluster.vx,cluster.vy,cluster.vz)
print('Fraction of stars with vtheta<0 =', np.sum(vtheta<0)/cluster.ntot)

#Now switch the sign of vtheta for 50% of stars with vtheta<0
print('Add rotation of 50%')
cluster.add_rotation(qrot=0.5)
r, theta, z, vr, vtheta, vz = ct.cart_to_cyl(cluster.x,cluster.y,cluster.z,cluster.vx,cluster.vy,cluster.vz)
print('Now fraction of stars with vtheta<0 =', np.sum(vtheta<0)/cluster.ntot)

In [ ]:
print('Half-Mass Relaxation Time: ',cluster.half_mass_relaxation_time())
print('Core Relaxation Time: ',cluster.core_relaxation_time())
print('Lagrange Radii: ',cluster.rlagrange())
print('Virial Radius: ',cluster.virial_radius())
print("projected:")
print('Half-Mass Relaxation Time: ',cluster.half_mass_relaxation_time(projected=True))
print('Core Relaxation Time: ',cluster.core_relaxation_time(projected=True))
print('Lagrange Radii: ',cluster.rlagrange(projected=True))
print('Virial Radius: ',cluster.virial_radius(projected=True))

In [ ]:
cluster.rcore(plot=True)
print(cluster.rc)

- Hey, no units?

In [ ]:
m_mean, m_hist, dm, alpha, ealpha, yalpha, eyalpha = ct.mass_function(cluster,mmin=0.1,mmax=0.8,plot=True)


- Fair enough, the stars are equal mass

In [ ]:
m_mean, sigvm, eta, eeta, yeta, eyeta = ct.eta_function(cluster,plot=True)

- again, fair enough

In [ ]:
from galpy.potential import MWPotential2014

print('Tidal Radius: ', cluster.rtidal(pot=MWPotential2014))
print('Limiting Radius: ', cluster.rlimiting(pot=MWPotential2014,plot=True))

In [ ]:
distance_to_neighbour = ct.closest_star(cluster)
print(np.amin(distance_to_neighbour))

In [ ]:
rprof, pprof, nprof = ct.rho_prof(cluster,plot=True)

In [ ]:
rprof, mprof, nprof = ct.m_prof(cluster,plot=True)

In [ ]:
rprof, mprof, nprof = ct.m_prof(cluster,plot=True,cumulative=True)

In [ ]:
rprof, vcprof, rvmax, vmax = ct.vcirc_prof(cluster,plot=True,nrad=None)
print(rvmax,vmax)

In [ ]:
rprofn, sigvprof = ct.sigv_prof(cluster,plot=True)
rprofn_r, sigvprof_r = ct.sigv_prof(cluster,coord='r',plot=True)
rprofn_phi, sigvprof_phi = ct.sigv_prof(cluster,coord='phi',plot=True)
rprofn_theta, sigvprof_theta = ct.sigv_prof(cluster,coord='theta',plot=True)

In [ ]:
rprofn, betaprof = ct.beta_prof(cluster,plot=True)

- Line rather than scattered? Differs from documentation

In [ ]:
rprofn, vprof = ct.v_prof(cluster,plot=True)


In [ ]:
rprofn, aprof, dalpha, edalpha, ydalpha, eydalpha = ct.alpha_prof(cluster,mmin=0.1,mmax=0.8, plot=True, normalize=True)

In [ ]:
rprofn, eprof, deta, edeta, ydeta, eydeta = ct.eta_prof(cluster,plot=True, normalize=True)

In [ ]:
rprofn, meqprof, dmeq, edmeq, ydmeq, eydmeq = ct.meq_prof(cluster,plot=True, normalize=True)